# Mirrored Strategy 

Suitable for single machines with multiple devices (GPUs, CPUs, TPUs).
Creates a replica per device. Each variable is mirrored and then reduced across all devices.


2 key areas to change code for strategy:
- model declaration
- data preprocessing

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow import keras
import tensorflow_datasets as tfds

In [7]:
datasets, info = tfds.load(name="mnist", with_info=True, as_supervised=True)

In [8]:
mnist_train, mnist_test = datasets['train'], datasets['test']

In [9]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [18]:
# Number of devices help set up batch size for distributed training
print("Number of devices {}".format(strategy.num_replicas_in_sync))

Number of devices 1


In [11]:
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [12]:
def scale(image, label):
    '''
    normalise image by dividing by 255
    '''
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

In [14]:
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

In [15]:
# define model within scope of strategy
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation="relu", input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(10)
    ])

In [16]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy']
             )

In [17]:
model.fit(train_dataset, epochs=2)

Epoch 1/2
938/938 [==============================] - 18s 17ms/step - loss: 0.4078 - accuracy: 0.8860
Epoch 2/2
938/938 [==============================] - 13s 14ms/step - loss: 0.0771 - accuracy: 0.9776
